In [39]:
import numpy as np

In [40]:
"""число сочетаний из n по k"""
comb = lambda n, k: 0 if k > n else np.math.factorial(n) // (np.math.factorial(k) * np.math.factorial(n-k))

"""формула Пуассона"""
poisson = lambda k, l: (l ** k) * (np.exp(-l)) / np.math.factorial(k)

## Урок 2. Дискретные случайные величины. Закон распределения вероятностей. Биномиальный закон распределения. Распределение Пуассона.

**1. Контрольная работа состоит из пяти вопросов. На каждый вопрос приведено четыре варианта ответа, один из которых правильный. Составьте закон распределения числа правильных ответов при простом угадывании. Найдите M(X), D(X).**

Биномиальный закон распределения: 
$$P(X = k) = C_n^k p^k (1 - p)^{n - k}$$

Здесь событие $A$ - *на вопрос был дан правильный ответ*; $n = 5$ - общее число опытов (пять вопросов); $X$ - число наступления события $A$ в $n$ опытах (число правильных ответов на пять вопросов); $p = \frac{1}{4}$ - вероятность наступления события $A$ в каждом опыте.

Посчитаем вероятность каждого исхода:
$$P(X = 0) = C_5^0 \cdot {1\over4}^0 \cdot {3\over4}^5 = 1 \cdot {1\over4}^0 \cdot {3\over4}^5 = {243\over1024}$$
$$P(X = 1) = C_5^1 \cdot {1\over4}^1 \cdot {3\over4}^4 = 5 \cdot {1\over4}^1 \cdot {3\over4}^4 = {405\over1024}$$
$$P(X = 2) = C_5^2 \cdot {1\over4}^2 \cdot {3\over4}^3 = 10 \cdot {1\over4}^2 \cdot {3\over4}^3 = {270\over1024}$$
$$P(X = 3) = C_5^3 \cdot {1\over4}^3 \cdot {3\over4}^2 = 10 \cdot {1\over4}^3 \cdot {3\over4}^2 = {90\over1024}$$
$$P(X = 4) = C_5^4 \cdot {1\over4}^4 \cdot {3\over4}^1 = 5 \cdot {1\over4}^4 \cdot {3\over4}^1 = {15\over1024}$$
$$P(X = 5) = C_5^5 \cdot {1\over4}^5 \cdot {3\over4}^0 = 1 \cdot {1\over4}^5 \cdot {3\over4}^0 = {1\over1024}$$

Закон распределения случайной величины:

<table border="3">
    <tr>
        <th>$x$</th>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>3</td>
        <td>4</td>
        <td>5</td>
    </tr>
    <tr>
        <th>$P(X = x)$</th>
        <td>$\frac{32}{1024}$</td>
        <td>$\frac{405}{1024}$</td>
        <td>$\frac{270}{1024}$</td>
        <td>$\frac{90}{1024}$</td>
        <td>$\frac{15}{1024}$</td>
        <td>$\frac{1}{1024}$</td>
    </tr>
</table>



Математическое ожидание:
$$M(X) = np = 5\cdot{1\over4} = 1.25$$

Дисперсия:
$$D(X) = np(1 - p) = 5\cdot{1\over4}\cdot{3\over4} = 0.9375$$


**2. Пользователь получает в среднем 10 писем со спамом на свой почтовый ящик за сутки. Найти число N, такое, что с вероятностью 0.95 пользователь получит не более N писем со спамом за текущий день.**

Задача на распределение Пуассона с параметром $\lambda = 10$. Нужно найти такое минимальное $N$, при котором $\displaystyle\sum_{k = 0}^{N} P(X = k) \geq 0.95$. Решим задачу на питоне.

In [41]:
l = 10
k = 0
sum_p = 0
while sum_p < 0.95:
    sum_p += poisson(k, l)
    k += 1

N = k - 1
print(N)

15


In [42]:
# проверка
sum(poisson(i, l) for i in range(16)) 

0.9512595966960214

Посчитаем квантиль порядка 0,95 с помощью готовой функции.

In [43]:
from scipy import stats

In [44]:
stats.poisson.ppf(0.95, 10)

15.0

Ответы совпадают.

**Ответ: $N = 15$**

**3. Производятся выстрелы по мишени. Вероятность попадания в мишень при одном выстреле равна 0.01. Сколько выстрелов нужно сделать чтобы быть уверенным с вероятностью 0.9, что хотя бы 6 раз будет совершено попадание?**

***Подсказка. Здесь предстоит немножко покодить.***

Вероятность попасть в мишень $K$ раз за $N$ выстрелов определяется формулой
$$\displaystyle\sum_{k = K}^{N} C_N^k \cdot p^k \cdot (1-p)^{N-k}$$

Для нашей задачи $K=6$, $p=0.01$. Для ответа на вопрос, достаточно определить минимальное $N$, при котором
$$\displaystyle\sum_{k = 0}^{K-1} C_N^k \cdot p^k \cdot (1-p)^{N-k} \leq 0.1$$


Определим функцию `sum_comb(K, N, p)`, которая считает эту сумму и методом перебора найдём $N$.

In [45]:
"""сумма вероятностеей попасть в мишень от 0 до K не включительно при N выстрелах с вероятностью попасть при одном выстреле p"""
def sum_comb(K: int, N: int, p: float) -> float:
    return sum(comb(N, k) * (p**k) * ((1-p)**(N-k)) for k in range(K))

In [46]:
N = 0
while True:
    if (sum_comb(6, N, 0.01) <= 0.1): break
    N += 1
print(N)

926


**Ответ: $N = 926$**